In [1]:
import argparse
import numpy as np
from tqdm import tqdm
from earlystopping import EarlyStopping
import sys
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import nn
import torch
from data.seqgen import ClimateData
from net_params import convlstm_encoder_params, convlstm_decoder_params
from model import ED
from decoder import Decoder
from encoder import Encoder
import os
import datetime
from torch.utils.tensorboard import SummaryWriter
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print("x")
sys.argv = ['main.py']
# TIMESTAMP = str(datetime.datetime.now()).replace(" ", "")
TIMESTAMP = 'ref-0'
parser = argparse.ArgumentParser()
parser.add_argument('-clstm',
                    '--convlstm',
                    help='use convlstm as base cell',
                    action='store_true')
parser.add_argument('--batch_size',
                    default=1,
                    type=int,
                    help='mini-batch size')
parser.add_argument('-lr', default=1e-4, type=float, help='G learning rate')
parser.add_argument('-frames_input',
                    default=7,
                    type=int,
                    help='sum of input frames')
parser.add_argument('-frames_output',
                    default=1,
                    type=int,
                    help='sum of predict frames')
parser.add_argument('-epochs', default=4, type=int, help='sum of epochs')
args = parser.parse_args()

random_seed = 1996
np.random.seed(random_seed)
torch.manual_seed(random_seed)
if torch.cuda.device_count() > 1:
    torch.cuda.manual_seed_all(random_seed)
else:
    torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

save_dir = './save_model/' + TIMESTAMP

trainFolder = ClimateData(is_train=True,
                          root='data/',
                          n_frames_input=args.frames_input,
                          n_frames_output=args.frames_output)
validFolder = ClimateData(is_train=False,
                          root='data/',
                          n_frames_input=args.frames_input,
                          n_frames_output=args.frames_output)
trainLoader = torch.utils.data.DataLoader(trainFolder,
                                          batch_size=args.batch_size,
                                          shuffle=False)
validLoader = torch.utils.data.DataLoader(validFolder,
                                          batch_size=args.batch_size,
                                          shuffle=False)

encoder_params = convlstm_encoder_params
decoder_params = convlstm_decoder_params


C:\Users\chaud\AppData\Local\Programs\Python\Python39\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


x


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [3]:
encoder = Encoder(
    convlstm_encoder_params[0], convlstm_encoder_params[1]).cuda()
decoder = Decoder(
    convlstm_decoder_params[0], convlstm_decoder_params[1]).cuda()
model0 = ED(encoder, decoder).cuda()
model1 = ED(encoder, decoder).cuda()
model2 = ED(encoder, decoder).cuda()
model3 = ED(encoder, decoder).cuda()
model4 = ED(encoder, decoder).cuda()


In [4]:
checkpoint0 = torch.load(
    './save_model/factor-0/checkpoint_0_0.000064.pth.tar')
model0.load_state_dict(checkpoint0['state_dict'])
### now you can evaluate it
model0.eval()
checkpoint1 = torch.load(
    './save_model/factor-1/checkpoint_0_0.000051.pth.tar')
model1.load_state_dict(checkpoint1['state_dict'])
### now you can evaluate it
model1.eval()
checkpoint2 = torch.load(
    './save_model/factor-2/checkpoint_0_0.000533.pth.tar')
model2.load_state_dict(checkpoint2['state_dict'])
### now you can evaluate it
model2.eval()
checkpoint3 = torch.load(
    './save_model/factor-3/checkpoint_0_0.000020.pth.tar')
model3.load_state_dict(checkpoint3['state_dict'])
### now you can evaluate it
model3.eval()
checkpoint4 = torch.load(
    './save_model/factor-4/checkpoint_0_0.000021.pth.tar')
model4.load_state_dict(checkpoint4['state_dict'])
### now you can evaluate it
model4.eval()


ED(
  (encoder): Encoder(
    (stage1): Sequential(
      (conv1_leaky_1): Conv2d(5, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (leaky_conv1_leaky_1): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (rnn1): CLSTM_cell(
      (conv): Sequential(
        (0): Conv2d(80, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): GroupNorm(8, 256, eps=1e-05, affine=True)
      )
    )
    (stage2): Sequential(
      (conv2_leaky_1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (leaky_conv2_leaky_1): LeakyReLU(negative_slope=0.2, inplace=True)
    )
    (rnn2): CLSTM_cell(
      (conv): Sequential(
        (0): Conv2d(160, 384, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (1): GroupNorm(12, 384, eps=1e-05, affine=True)
      )
    )
    (stage3): Sequential(
      (conv3_leaky_1): Conv2d(96, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (leaky_conv3_leaky_1): LeakyReLU(negative_slope=0.2, inplace=True)

In [5]:
t = tqdm(validLoader, leave=False, total=len(validLoader))
for i, (idx, targetVar, inputVar, _, _) in enumerate(t):
    inputs = inputVar.to(device)  # B,S,C,H,W
    label = targetVar.to(device)  # B,S,C,H,W
    print("xtl")
    model0.load_state_dict(checkpoint0['state_dict'])
    ### now you can evaluate it
    model0.eval()
    pred0 = model0(inputs)  # B,S,C,H,W
    break


  0%|          | 0/100 [00:00<?, ?it/s]

xtl


In [6]:
pred0.shape

torch.Size([1, 1, 1, 73, 144])

In [7]:
np.save('predsf0', pred0.detach().cpu())


In [11]:
np.save('labelsf0', label.detach().cpu())


In [12]:
np.save('inputsf0', inputs.detach().cpu())


In [7]:
pred0

tensor([[[[[0.2623, 0.2648, 0.2672,  ..., 0.2656, 0.2660, 0.2638],
           [0.2649, 0.2670, 0.2688,  ..., 0.2662, 0.2668, 0.2673],
           [0.2671, 0.2684, 0.2699,  ..., 0.2683, 0.2677, 0.2685],
           ...,
           [0.2278, 0.2275, 0.2280,  ..., 0.2291, 0.2280, 0.2279],
           [0.2278, 0.2267, 0.2260,  ..., 0.2273, 0.2276, 0.2263],
           [0.2264, 0.2259, 0.2256,  ..., 0.2248, 0.2247, 0.2260]]]]],
       device='cuda:0', grad_fn=<TransposeBackward0>)